In [1]:
pip install dash

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 13.2 MB/s eta 0:00:00


In [2]:
import requests
import dash
from dash import dcc, html
from dash.dependencies import Input, Output

# Function to retrieve weather forecast data
def fetch_weather_forecast(location, apikey):
    url = f"https://api.tomorrow.io/v4/weather/forecast?location={location}&apikey={apikey}"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        return data
    else:
        print("Failed to retrieve weather forecast data")
        return None

# Coordinates for Kansas City, Missouri
location = "39.0997,-94.5786"

# Your Tomorrow.io API key
apikey = "Y1bTFtuUW6fyE0q7t3hGR7cDRyAHrs4p"

# Fetch initial weather forecast data
initial_data = fetch_weather_forecast(location, apikey)

# Initialize the Dash app
app = dash.Dash(__name__)

# Define the layout of the dashboard
app.layout = html.Div([
    html.H1("Weather Forecast Dashboard"),
    html.Div(id='weather-display'),
    dcc.Interval(
        id='interval-component',
        interval=3600*1000,  # Update every hour
        n_intervals=0
    )
])

# Callback to update weather forecast data every hour
@app.callback(
    Output('weather-display', 'children'),
    [Input('interval-component', 'n_intervals')]
)
def update_weather_data(n):
    data = fetch_weather_forecast(location, apikey)
    if data and 'timelines' in data and 'minutely' in data['timelines']:
        minutely_forecast = data['timelines']['minutely']
        forecast_text = "Minutely Forecast:\n"
        for entry in minutely_forecast:
            forecast_text += f"{entry['time']}: {entry['values']['temperature']}°C\n"
        return html.Pre(forecast_text)
    else:
        print("Received unexpected data format from the API:", data)
        return "Failed to retrieve weather forecast data. Please try again later."

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)

<IPython.core.display.Javascript object>